In [235]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
df = pd.read_csv("head.csv")
clf_label = "sentiment_category" # can be changed with topic_category
X = df.the_document_tokens
if(clf_label == "sentiment_category"):
    y = df.sentiment_category
elif(clf_label == "topic_category"):
    y = df.topic_category


In [236]:
print(type(X))
print(type(y))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


In [237]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20)
print(type(X_train))
type(X_train)

<class 'pandas.core.series.Series'>


pandas.core.series.Series

In [188]:
#vectorizer = CountVectorizer(stop_words='english')
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train) 
X_test = vectorizer.transform(X_test)

In [189]:
feature_names = vectorizer.get_feature_names()
len(feature_names)
#len(vectorizer.vocabulary_)

42223

In [190]:
X_train = pd.DataFrame(X_train.toarray(), columns=feature_names) # converting Series to Dataframe
X_test = pd.DataFrame(X_test.toarray(), columns=feature_names) # converting Series to Dataframe

# Understanding the data

In [191]:
X_train[clf_label+"_label"] = np.array(y_train, dtype='object')

In [192]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9531 entries, 0 to 9530
Columns: 42224 entries, 00 to sentiment_category_label
dtypes: int64(42223), object(1)
memory usage: 3.0+ GB
None


In [193]:
categories = [category for category in X_train.sentiment_category_label.unique()]

In [194]:
X_trains = {}
for sent_category in categories:
    X_trains[sent_category] = X_train[X_train['sentiment_category_label']==sent_category]
    X_trains[sent_category] = X_trains[sent_category].iloc[: , :-1]


In [195]:
len(X_trains["neg"]), len(X_trains["pos"]) # number of documents in negative and positive cases in training data

(4723, 4808)

In [ ]:
max_freq_in_negative_case = {} # created to store most freqeunt words in negative labeled documents
for word in X_trains["neg"]:
    freq = X_trains["neg"][word].sum() # sum of the specific word in negative case 
    max_freq_in_negative_case[word] = freq

In [ ]:
max_freq_in_negative_case = dict(sorted(max_freq_in_negative_case.items(), key=lambda item: item[1], reverse=True))
count = 1
for key in max_freq_in_negative_case:
    print(key,"->freq: ",max_freq_in_negative_case[key])
    if(count == 10): break
    count+=1

In [ ]:
max_freq_in_positive_case = {} # created to store most freqeunt words in positive labeled documents
for word in X_trains["pos"]:
    freq = X_trains["pos"][word].sum() # sum of the specific word in positive case 
    max_freq_in_positive_case[word] = freq

In [ ]:
max_freq_in_positive_case = dict(sorted(max_freq_in_positive_case.items(), key=lambda item: item[1], reverse=True)) 
count = 1
for key in max_freq_in_positive_case:
    print(key,"->freq: ",max_freq_in_positive_case[key])
    if(count == 10): break
    count+=1

In [164]:
difference = {} # to store freqeuency difference of common words in positive and negative cases
for key in max_freq_in_negative_case.keys(): 
    difference[key] = abs(max_freq_in_positive_case[key] - max_freq_in_negative_case[key])

In [165]:
difference = dict(sorted(difference.items(), key=lambda item: item[1], reverse=True))
count = 1
for key in difference:
    print(key,"->freq: ",difference[key])
    if(count == 10): break
    count+=1

and ->freq:  2460
not ->freq:  1922
was ->freq:  1397
is ->freq:  1313
great ->freq:  1153
of ->freq:  1133
that ->freq:  988
the ->freq:  980
to ->freq:  838
as ->freq:  818


In [166]:
y_train = np.array(y_train, dtype='object')
y_test = np.array(y_test, dtype='object')

In [167]:
print(y_train)

['books' 'dvd' 'health' ... 'health' 'dvd' 'software']


# Implementing Naive Bayes

In [168]:
# Naive Bayes Classifier (Works for both sentiment category and topic category)
class NaiveBayesClassifier: 
    # outcomesPD is X, y is y_train -> dtype = 'object'
    def fit(self, X, y_train, clf_label):
        (unique, counts) = np.unique(y_train, return_counts=True) # count the unique number of classes in the category label
        frequencies = np.asarray((unique, counts)).T 
        self.freq = {} # to store the frequency of the categories seperately
        for category, frequency in frequencies:
            self.freq[category] = frequency  # store the freqeuncy for each category
        total_examples = sum(self.freq.values()) # total number of documents in the dataset
        prob_category = {} # to store the category probabilities
        for category in self.freq.keys():
            prob_category[category] = self.freq[category] / total_examples # to calculate the probability of the category class
        print(prob_category) # print the probabilities of each category seperately
        number_of_word_in_category_case = {} # to store the number of word in the specific category
        outcomesCategoryPD = {} # will be used in filtering the categories from the training data
        prob_word_given_category = {} # category, word
        for category in self.freq.keys(): 
            outcomesCategoryPD[category] = X[X[clf_label]==category] # filtering operation
            number_of_word_in_category_case[category] = outcomesCategoryPD[category].iloc[: , :-1].values.sum() # total number of word is specific category case
            prob_word_given_category[category] = {} # to store the conditional probability of each word for each class
        vocabularies = [word for word in X.iloc[: , :-1]] # vocabularies in training data
        vocab_size = len(vocabularies) # vocabulary size
        prior_category = {} # to store the prior probability of any category
        posterior_category = {} # to store the posterior information for each category
        for category in self.freq.keys():
            for word in vocabularies:
                count_specific_word_in_category_case = outcomesCategoryPD[category][word].sum() + 1 # count the specific word in the specific category
                prob_word_given_category[category][word] = np.log(count_specific_word_in_category_case / (number_of_word_in_category_case[category]+vocab_size)) # calculate the conditional probability using Laplace Smooting
                prior_category[category] = np.log(prob_category[category]) # store the prior probability for each category
                posterior_category[category] = prob_word_given_category[category][word] + prior_category[category]
        return prior_category, prob_word_given_category,vocabularies,outcomesCategoryPD # return the prior category, conditional probabilities, vocabularies
    
    def predict(self, X): # X is X_test -> outcomesTestPD
        test_views = []
        for test_index in X.index:
            test_features = X.loc[test_index]
            test_features = test_features[test_features > 0]
            test_view = [feature for feature in test_features.to_frame().T.columns]
            test_views.append(test_view)
        test_views
        print(len(test_views[0]))
        y_pred = np.array([]).astype(np.object)
        for view in test_views:
            posterior_for_category = {}
            for category in self.freq.keys():
                posterior_for_category[category] = prior_category[category]
                for token in view:
                    if(token in vocabularies):
                        posterior_for_category[category] += prob_word_given_category[category][token] # sum all the log likelihood with the prior probability 
            max_key = max(posterior_for_category, key=posterior_for_category.get) # return argmax of the predicted class
            y_pred = np.append(y_pred, max_key) # append the predicted class to y_pred 
        return y_pred # return y_pred

In [169]:
def calculate_accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

In [170]:
nb_clf_for_categories = NaiveBayesClassifier()

In [171]:
prior_category, prob_word_given_category,vocabularies,outcomesCategoryPD = nb_clf_for_categories.fit(X_train,y_train,clf_label+"_label")

{'books': 0.16608960235022557, 'camera': 0.16692896862868534, 'dvd': 0.16913230510964222, 'health': 0.16703388941349281, 'music': 0.1685027804007974, 'software': 0.16231245409715664}


In [172]:
y_pred = nb_clf_for_categories.predict(X_test)

174


In [173]:
accuracy = calculate_accuracy(y_test, y_pred)

In [174]:
print(accuracy)

0.9068401174989509


In [177]:
#prob_word_given_category["neg"]["like"], prob_word_given_category["pos"]["like"]
#prob_word_given_category["neg"]["book"], prob_word_given_category["pos"]["book"]
cond_neg = dict(sorted(prob_word_given_category["books"].items(), key=lambda item: item[1], reverse=True))
count = 1
for key in cond_neg:
    print("P(word='{}'|{}) = {}".format(key,"label = neg",cond_neg[key]))
    if(count == 10): break
    count+=1

P(word='the'|label = neg) = -3.039581590486498
P(word='of'|label = neg) = -3.6407692653538737
P(word='and'|label = neg) = -3.6599966934410935
P(word='to'|label = neg) = -3.7279279937030863
P(word='is'|label = neg) = -4.138712263288851
P(word='in'|label = neg) = -4.21415445845571
P(word='this'|label = neg) = -4.377251500081034
P(word='it'|label = neg) = -4.385181306573279
P(word='book'|label = neg) = -4.451598190103922
P(word='that'|label = neg) = -4.483082272037205


In [178]:
cond_pos = dict(sorted(prob_word_given_category["books"].items(), key=lambda item: item[1], reverse=True))
count = 1
for key in cond_pos:
    print("P(word='{}'|{}) = {}".format(key,"label = pos",cond_pos[key]))
    if(count == 10): break
    count+=1

P(word='the'|label = pos) = -3.039581590486498
P(word='of'|label = pos) = -3.6407692653538737
P(word='and'|label = pos) = -3.6599966934410935
P(word='to'|label = pos) = -3.7279279937030863
P(word='is'|label = pos) = -4.138712263288851
P(word='in'|label = pos) = -4.21415445845571
P(word='this'|label = pos) = -4.377251500081034
P(word='it'|label = pos) = -4.385181306573279
P(word='book'|label = pos) = -4.451598190103922
P(word='that'|label = pos) = -4.483082272037205


In [179]:
def printConfusionMatrix(y_pred, y_test):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [180]:
from sklearn.metrics import classification_report, confusion_matrix
printConfusionMatrix(y_pred, y_test)

[[362   1  35   3   3  13]
 [  0 395   0   5   1   7]
 [ 14  10 332   4  24   4]
 [  1  34   0 354   1  18]
 [  0   3  15   0 371   5]
 [  4   6   5   2   4 347]]
              precision    recall  f1-score   support

       books       0.95      0.87      0.91       417
      camera       0.88      0.97      0.92       408
         dvd       0.86      0.86      0.86       388
      health       0.96      0.87      0.91       408
       music       0.92      0.94      0.93       394
    software       0.88      0.94      0.91       368

    accuracy                           0.91      2383
   macro avg       0.91      0.91      0.91      2383
weighted avg       0.91      0.91      0.91      2383



In [181]:
y_train,y_test

(array(['books', 'dvd', 'health', ..., 'health', 'dvd', 'software'],
       dtype=object),
 array(['dvd', 'music', 'books', ..., 'books', 'health', 'books'],
       dtype=object))

# Calculate Accuracy With The Sentiment Categories

In [196]:
nb_clf_for_categories = NaiveBayesClassifier()

In [197]:
prior_category, prob_word_given_category,vocabularies,outcomesCategoryPD = nb_clf_for_categories.fit(X_train,y_train,clf_label+"_label")

{'neg': 0.49554086664568253, 'pos': 0.5044591333543175}


In [198]:
y_pred = nb_clf_for_categories.predict(X_test)

262


In [199]:
accuracy = calculate_accuracy(y_test, y_pred)

In [200]:
print(accuracy)

0.819555182543013


In [201]:
def printConfusionMatrix(y_pred, y_test):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [202]:
from sklearn.metrics import classification_report, confusion_matrix
printConfusionMatrix(y_pred, y_test)

[[1027  164]
 [ 266  926]]
              precision    recall  f1-score   support

         neg       0.79      0.86      0.83      1191
         pos       0.85      0.78      0.81      1192

    accuracy                           0.82      2383
   macro avg       0.82      0.82      0.82      2383
weighted avg       0.82      0.82      0.82      2383



In [ ]:
y_train,y_test

# TF-IDF (using TfidfVectorizer)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
vectorizer = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS)

In [ ]:
X_train = vectorizer.fit_transform(X_train) 
X_test = vectorizer.transform(X_test)

In [ ]:
feature_names = vectorizer.get_feature_names()
print(len(feature_names))

In [ ]:
X_train = pd.DataFrame(X_train.toarray(), columns=feature_names)
X_test = pd.DataFrame(X_test.toarray(), columns=feature_names)

In [ ]:
X_train[clf_label+"_label"] = np.array(y_train, dtype='object')

In [ ]:
y_train = np.array(y_train, dtype='object')
y_test = np.array(y_test, dtype='object')

In [ ]:
nb = NaiveBayesClassifier()

In [ ]:
prior_category, prob_word_given_category,vocabularies,outcomesCategoryPD = nb.fit(X_train,y_train,clf_label+"_label")

In [ ]:
y_pred = nb.predict(X_test)

In [203]:
accuracy = calculate_accuracy(y_test, y_pred)

In [204]:
print(accuracy)

0.819555182543013


In [205]:
most_strong_words_in_positive = dict(sorted(prob_word_given_category["pos"].items(), key=lambda item: item[1], reverse=True))
most_strong_words_in_negative = dict(sorted(prob_word_given_category["neg"].items(), key=lambda item: item[1], reverse=True))

In [206]:
count = 1
for key in most_strong_words_in_negative:
    print("P(word='{}'|{}) = {}".format(key,"label = neg",most_strong_words_in_negative[key]))
    if(count == 10): break
    count+=1

P(word='the'|label = neg) = -2.986205259442668
P(word='to'|label = neg) = -3.6395180852701943
P(word='and'|label = neg) = -3.674097760937825
P(word='of'|label = neg) = -3.889243467489403
P(word='it'|label = neg) = -3.929729803339705
P(word='is'|label = neg) = -4.104418698339079
P(word='this'|label = neg) = -4.183943112599831
P(word='that'|label = neg) = -4.380076703002806
P(word='in'|label = neg) = -4.405225771423476
P(word='for'|label = neg) = -4.652111317653339


In [207]:
count = 1
for key in most_strong_words_in_positive:
    print("P(word='{}'|{}) = {}".format(key,"label = pos",most_strong_words_in_positive[key]))
    if(count == 10): break
    count+=1

P(word='the'|label = pos) = -2.948732535743196
P(word='and'|label = pos) = -3.530831159849995
P(word='to'|label = pos) = -3.697052407444448
P(word='of'|label = pos) = -3.8130752113016464
P(word='is'|label = pos) = -3.987906518225581
P(word='it'|label = pos) = -3.990856497186509
P(word='this'|label = pos) = -4.232056922719085
P(word='in'|label = pos) = -4.300068018340222
P(word='that'|label = pos) = -4.515308201100143
P(word='for'|label = pos) = -4.564112626637668


In [208]:
def printConfusionMatrix(y_pred, y_test):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [209]:
from sklearn.metrics import classification_report, confusion_matrix
printConfusionMatrix(y_pred, y_test)

[[1027  164]
 [ 266  926]]
              precision    recall  f1-score   support

         neg       0.79      0.86      0.83      1191
         pos       0.85      0.78      0.81      1192

    accuracy                           0.82      2383
   macro avg       0.82      0.82      0.82      2383
weighted avg       0.82      0.82      0.82      2383



# TF-IDF (using CountVectorizer)

In [213]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text  import TfidfTransformer

In [214]:
vectorizer = CountVectorizer(stop_words=ENGLISH_STOP_WORDS)  #instantiate CountVectorizer()

In [215]:
word_count_vector = vectorizer.fit_transform(X_train) # this steps generates word counts for the words in the documets 
#X_test = vectorizer.transform(X_test)

In [216]:
#Compute the IDF values
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [217]:
#tfidf_transformer.fit(word_count_vector)

In [218]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=vectorizer.get_feature_names(),columns=["idf_weights"])

In [219]:
# sort ascending 
df_idf.sort_values(by=['idf_weights']).head() #Notice that the words ‘just’ and ‘like’ have the lowest IDF values. This is expected as these words appear in each and every document in our collection. The lower the IDF value of a word, the less unique it is to any particular document.

,idf_weights
like,2.283118
just,2.303383
good,2.485473
great,2.506073
time,2.568532


In [220]:
#  Compute the TFIDF score for the documents
X_train=vectorizer.transform(X_train) 
 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(X_train) # to compute the tf-idf scores for documents

In [221]:
feature_names = vectorizer.get_feature_names()  # get feature names
first_document_vector=tf_idf_vector[0] #get tfidf vector for first document 
shape = first_document_vector.shape
tf_idf_vector

<9531x41571 sparse matrix of type '<class 'numpy.float64'>'
	with 461832 stored elements in Compressed Sparse Row format>

In [222]:
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False).head()

,tfidf
onions,0.761914
cut,0.214762
hahhahaha,0.205523
onion,0.190478
chopping,0.185635


In [223]:
sorted_vals = df.sort_values(by=["tfidf"],ascending=False)
most_weighted_words = {}
for index in sorted_vals.index:
    features = sorted_vals.loc[index]
    if(features["tfidf"] > 0):
        most_weighted_words[features.name] = features["tfidf"]
print(most_weighted_words.keys())

dict_keys(['onions', 'cut', 'hahhahaha', 'onion', 'chopping', 'carrot', 'coz', 'vegetables', 'bought', 'tears', 'dont', 'broke', 'soft', 'suggest', 'mentioned', '15', 'save', 'buying', 'days', 'probably', 'buy', 'great', 'good'])


In [224]:
X_train = pd.DataFrame(X_train.toarray(), columns=feature_names) #convert series to dataframe

In [225]:
X_train[clf_label+"_label"] = np.array(y_train, dtype='object')

In [226]:
y_train = np.array(y_train, dtype='object')
y_test = np.array(y_test, dtype='object')

In [227]:
nb = NaiveBayesClassifier()
X_train.head()

,00,000,0003,000mb,004144,007,00am,00pm,01,02,...,½kerlund,½lan,½sistance,½ttbullar,½ve,áron,árpád,ça,único,sentiment_category_label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,neg
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pos
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,neg
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pos
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,neg


In [ ]:
prior_category, prob_word_given_category,vocabularies,outcomesCategoryPD = nb.fit(X_train,y_train,clf_label+"_label")

In [ ]:
X_test = vectorizer.transform(X_test)

In [ ]:
X_test = pd.DataFrame(X_test.toarray(), columns=feature_names)
X_test.head()

In [ ]:
y_pred = nb.predict(X_test)

In [228]:
accuracy = calculate_accuracy(y_test, y_pred)

In [234]:
print(accuracy)

0.8196810742761225


# Bigram

In [238]:
vectorizer = CountVectorizer(stop_words=ENGLISH_STOP_WORDS,ngram_range=(2, 2))
X_train = vectorizer.fit_transform(X_train) 
X_test = vectorizer.transform(X_test)

In [239]:
feature_names = vectorizer.get_feature_names()
len(feature_names)

428145

In [ ]:
X_train = pd.DataFrame(X_train.toarray(), columns=feature_names)
X_test = pd.DataFrame(X_test.toarray(), columns=feature_names)

In [ ]:
X_train

In [ ]:
X_train = X_train.iloc[:,:42031]
X_train[clf_label+"_label"] = np.array(y_train, dtype='object')

In [ ]:
X_test = X_test.iloc[:,:42031]
X_test

In [ ]:
y_train = np.array(y_train, dtype='object')
y_test = np.array(y_test, dtype='object')
X_train

In [ ]:
nb_clf_for_categories = NaiveBayesClassifier()

In [ ]:
prior_category, prob_word_given_category,vocabularies,outcomesCategoryPD = nb_clf_for_categories.fit(X_train,y_train,clf_label+"_label")

In [ ]:
y_pred = nb_clf_for_categories.predict(X_test)

In [ ]:
accuracy = calculate_accuracy(y_test, y_pred)

In [ ]:
print(accuracy)

In [ ]:
def printConfusionMatrix(y_pred, y_test):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
printConfusionMatrix(y_pred, y_test)

In [ ]:
cond_pos = dict(sorted(prob_word_given_category["pos"].items(), key=lambda item: item[1], reverse=True))
count = 1
for key in cond_pos:
    print("P(word='{}'|{}) = {}".format(key,"label = pos",cond_pos[key]))
    if(count == 10): break
    count+=1

In [ ]:
cond_neg = dict(sorted(prob_word_given_category["neg"].items(), key=lambda item: item[1], reverse=True))
count = 1
for key in cond_neg:
    print("P(word='{}'|{}) = {}".format(key,"label = neg",cond_neg[key]))
    if(count == 10): break
    count+=1